This notebook outlines the methods used to generate the models discussed in the CARD:Shark 3 paper.

# Parameter(s)

- `SUBSAMPLING` (bool): adjust if you would like to subsample the training/testing data to make them balanced

In [ ]:
# Adjust this if you would like to subsample the data
SUBSAMPLING = True

In [ ]:
# Importing modules
import json
import os
import sys
import pickle
import re
import pandas as pd
import numpy as np

from sklearn import (linear_model,
                     naive_bayes,
                     ensemble,
                     svm,
                     model_selection,
                     metrics,
                     model_selection)
import xgboost
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from numpy import interp
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Paper loading and processing

In [ ]:
# Load positive and negative papers
def process_text(df):
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    processed_text = []
    for column in ['text']:
        for i, sentence in enumerate(df[column]):
            # Convert to lower-case
            sentence = sentence.lower()

            # Removing punctuation
            sentence = re.sub(r'[?|!|\'|"|#]', r'', sentence)
            sentence = re.sub(r'[.|,|)|(|\|/]', r' ', sentence)

            # Removing stop words and stemming
            stemmer = PorterStemmer()
            words = [stemmer.stem(word) for word in sentence.split(
            ) if word not in stopwords.words('english')]

            # Removing digits
            words = [word for word in words if not word.isdigit()]

            processed_text.append(' '.join(words))

        df['processed_%s' % column] = processed_text

    return df


card_dataset_df_file = "card_papers.pkl"

with open(card_dataset_df_file, 'rb') as file:
    df = pickle.load(file)

if "processed_text" not in df:
    df = process_text(df)
    with open(card_dataset_df_file, 'wb') as file:
        pickle.dump(df, file)
df.head()

## Subsampling text
To test subsampling, change `SUBSAMPLING` to `True`

In [ ]:
# Used to test subsampling
if SUBSAMPLING:
    BASE_PATH = os.path.join('results', 'subsampled')
    df = df.loc[df['label'] == 0].sample(1000, random_state=100).append(df.loc[df['label'] == 1].sample(1000, random_state=100))
else:
    BASE_PATH = os.path.join('results', 'non-subsampled')

if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)

In [ ]:
df['label'].value_counts()

# Class used to organize models

In [ ]:
# Class that contains the function used to organize ml models
class ML:
    def __init__(self, model, model_name, feature, feature_name, processed_text, parameters={}):
        self.model = model
        self.model_name = model_name
        self.feature = feature
        self.feature_name = feature_name
        self.parameters = parameters
        self.pipeline = None
        self.stats = {}
        self.processed_text = processed_text

    def create_pipeline(self):
        pipeline = Pipeline(
            self.feature +
            [(self.model_name, self.model)]
        )

        param_keys = pipeline.get_params()
        params = dict(filter(lambda x: x[0] in param_keys,
                             self.parameters.items()))
        pipeline.set_params(**params)
        self.pipeline = pipeline

    def set_pipeline(self, X, y):
        return self.pipeline.fit(X, y)

    def get_stats(self):
        return self.stats
        
    def get_pipeline(self):
        return self.pipeline

def cross_validate(model, X_cv, y_cv, seed=100, splits=5):
    cv = model_selection.StratifiedKFold(
        n_splits=splits, random_state=seed, shuffle=True)

    cv_results = {}
    for i, (train_index, test_index) in enumerate(cv.split(X_cv, y_cv)):

        pipeline = model.pipeline.fit(X_cv[train_index], y_cv[train_index])

        probas_ = pipeline.predict_proba(X_cv[test_index])
        fpr, tpr, thresholds = metrics.roc_curve(
            y_cv[test_index], probas_[:, 1])
        auc = metrics.auc(fpr, tpr)
        precision, recall, _ = metrics.precision_recall_curve(
            y_cv[test_index], probas_[:, 1])

        predictions = pipeline.predict(X_cv[test_index])
        cm = metrics.confusion_matrix(y_cv[test_index], predictions)
        mse = metrics.mean_squared_error(y_cv[test_index], predictions)
        cv_results[i] = dict(
            fpr=fpr,
            tpr=tpr,
            thresholds=thresholds,
            precision=precision,
            recall=recall,
            cm=cm,
            auc=auc,
            mse=mse
        )

    # Get final stats
    mean_fpr = np.linspace(0, 1, 100)
    aucs = []
    tprs = []
    for _, result in cv_results.items():
        auc = metrics.auc(result['fpr'], result['tpr'])
        aucs.append(auc)
        tpr = interp(mean_fpr, result['fpr'], result['tpr'])
        tpr[0] = 0.0
        tprs.append(tpr)

    mean_tpr = np.mean(tprs, axis=0)
    std = np.std(tprs, axis=0)
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    tprs_upper = np.minimum(mean_tpr+std, 1)
    tprs_lower = mean_tpr - std


    cv_results['total'] = dict(
        mean_auc=mean_auc,
        std=std,
        tprs_upper=tprs_upper,
        tprs_lower=tprs_lower,
        std_auc=std_auc,
        mean_fpr=mean_fpr,
        mean_tpr=mean_tpr
    )
    model.stats = cv_results
    return model

## Creating combinations between model and feature extraction methods

In [ ]:
import itertools

MODELS = dict(
    lr=linear_model.LogisticRegression(random_state=100),
    nb=naive_bayes.MultinomialNB(),
    rf=ensemble.RandomForestClassifier(random_state=100),
    xgb=xgboost.XGBClassifier(),
    svm=svm.SVC(probability=True)
).items()

FEATURES = dict(
    count_word=[('count-word', CountVectorizer())],
    tf_word=[
        ('count-word', CountVectorizer()),
        ('tf-idf', TfidfTransformer())],
    tf_bitrigram=[
        ('count-word', CountVectorizer(ngram_range=(2,3))),
        ('tf-idf', TfidfTransformer())]
    ).items()

combinations = list(itertools.product(MODELS, FEATURES))

print('There are %s different combinations to examine' % len(combinations))

# Functions for cross-validation

In [ ]:

def get_splits(df, processed_text):
    if processed_text:
        papers = df['processed_text'].values
    else:
        papers = df['text'].values
    labels = df['label'].values.astype("int32")
    X_cv, X_holdout, y_cv, y_holdout = model_selection.train_test_split(
                papers, labels, random_state=100)
    return [X_cv, X_holdout, y_cv, y_holdout]

def run_pipelines(splits, processed_text):
    ml_pipelines = []
    X_cv, X_holdout, y_cv, y_holdout = splits
    for ((model_name, model_fx), (feature_name, feature_fx)) in tqdm(combinations, desc="Models (Processed text: %s)" % (processed_text)):
        ml = ML(model_fx, model_name, feature_fx, feature_name, processed_text)
        ml.create_pipeline()
        ml = cross_validate(ml, X_cv, y_cv)
        ml_pipelines.append(ml)
    return ml_pipelines

def create_models():
    unprocessed_splits = get_splits(df, False)
    processed_splits = get_splits(df, True)
    return run_pipelines(unprocessed_splits, False) + run_pipelines(processed_splits, True)

# CARD:Shark 2 methods

In [ ]:
import card_shark_functions 
from card_shark_functions import *

def run_card_shark():
    seed = 100 
    exclude = ["in", "and", "the", "or", "if", "a", "of", "from", "by", "with"]
    text_processed = [False, True]

    cv = model_selection.StratifiedKFold(
            n_splits=5, random_state=100, shuffle=True)

    for processed in text_processed:
        ml = ML(None, "card_shark", None, None, processed)
        cv_results = {}
        (cardWords, cardJournals, cardAbstracts) = wordFrequency(df.loc[df['label']==1], exclude, process=processed) 

        X_cv, _, y_cv, _ = get_splits(df, processed)
        for i, (train_index, test_index) in enumerate(tqdm(list(cv.split(X_cv, y_cv)), desc="CARD:Shark (Processed text: %s)" % (processed))):
            df_ = []
            for z in X_cv:
                if processed:
                    df_.append(df.loc[df['processed_text'] == z].copy())
                else:
                    df_.append(df.loc[df['text'] == z].copy())
            df_ = pd.concat(df_, axis=0)
            df_ = df_.reset_index()
            df_train = df_.iloc[train_index]
            df_test = df_.iloc[test_index]

            (sampleWords, sampleJournals, sampleAbstracts) = wordFrequency(df_train, exclude, process=processed)
            matrixA = matrixMaker(cardWords, sampleWords, True)
            matrixD = doubleMatrixMaker(cardAbstracts, sampleAbstracts, exclude)
            matrixJ = matrixMaker(cardJournals, sampleJournals, True)
            stats, _ = bluePill('text_pmids', matrixA, matrixJ, matrixD, exclude, df_test, processed)
            cv_results[i] = stats


        mean_fpr = np.linspace(0, 1, 100)
        aucs = []
        tprs = []
        for _, result in cv_results.items():
            auc = metrics.auc(result['fpr'], result['tpr'])
            aucs.append(auc)
            tpr = interp(mean_fpr, result['fpr'], result['tpr'])
            tpr[0] = 0.0
            tprs.append(tpr)

        mean_tpr = np.mean(tprs, axis=0)
        std = np.std(tprs, axis=0)
        mean_auc = metrics.auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)

        tprs_upper = np.minimum(mean_tpr+std, 1)
        tprs_lower = mean_tpr - std

        cv_results['total'] = dict(
            mean_auc=mean_auc,
            std=std,
            tprs_upper=tprs_upper,
            tprs_lower=tprs_lower,
            std_auc=std_auc,
            mean_fpr=mean_fpr,
            mean_tpr=mean_tpr
        )
        ml.stats = cv_results
        yield ml

# Running cross-validation on CARD:Shark 2 and 3

Loads pipelines if they exist, otherwise creates and saves them

In [ ]:
PIPELINE_PATH = os.path.join(BASE_PATH, 'all_pipelines.pkl')
try:
    with open(PIPELINE_PATH, 'rb') as file:
        all_pipelines = pickle.load(file)
except:
    all_pipelines = create_models() + [x for x in run_card_shark()]
    with open(PIPELINE_PATH, 'wb') as file:
        pickle.dump(all_pipelines, file)

# Visualization

In [ ]:
c = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
colors = dict(
    nb=c[0],
    lr=c[1],
    rf=c[2],
    xgb=c[3],
    svm=c[4],
    card_shark=c[5]
)
names = dict(
    nb="Naive Bayes",
    lr="Logistic regression",
    rf="Random forest",
    xgb="Extreme gradient boosting",
    svm="Support vector machine",
    card_shark="CARD:Shark 2"
)
features = dict(
    count_word="Word count",
    tf_word="TF-IDF Word",
    tf_bitrigram="TF-IDF Bi/tri-gram"
)
lines = dict(
    count_word="solid",
    tf_word="dotted",
    tf_bitrigram="dashed"
)

In [ ]:
def create_roc(ml_items, process, save=False):
    plt.clf()
    fig = plt.figure(dpi=300)
    fig.set_figwidth(9)
    fig.set_figheight(8)
    ax = plt.axes(xlim=(-0.05,1.05), ylim=(-0.05,1.05))
    plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])
    plt.yticks([0.0,0.2,0.4,0.6,0.8,1.0])

    ax.plot([0,1], [0,1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    for ml in ml_items:
        if ml.processed_text != process:
            continue
        model_name = names.get(ml.model_name)
        feature_name = features.get(ml.feature_name)
        color = colors.get(ml.model_name)
        line = lines.get(ml.feature_name)
        stats = ml.stats['total']
        mean_fpr = stats['mean_fpr']
        mean_tpr = stats['mean_tpr']
        mean_auc = stats['mean_auc']
        ax.plot(mean_fpr, mean_tpr, alpha=0.9, label=r'Mean ROC %s, %s (AUC = %0.2f)' % (model_name, feature_name, mean_auc), color=color, linestyle=line)
        ax.fill_between(mean_fpr, stats['tprs_lower'], stats['tprs_upper'], alpha = 0.2, color=color)

    ax.legend(loc="lower right")
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')

    if save:
        plt.savefig(save+'.png')
        plt.savefig(save+'.svg', format="svg")

    plt.show()

print("Unprocessed text ROC")
create_roc(all_pipelines, False, os.path.join(BASE_PATH, 'unprocessed_text_roc'))
print("Processed text ROC")
create_roc(all_pipelines, True, os.path.join(BASE_PATH, 'processed_text_roc'))

In [ ]:
def create_table(all_pipelines, save=False):
    results = []
    for ml in all_pipelines:
        cm = sum([y['cm'].ravel() for x,y in ml.stats.items() if x != 'total'])
        precision = cm[3] / (cm[3]+cm[1])
        recall = cm[3] / (cm[3]+cm[2])
        f1 = 2*precision*recall/(precision+recall)
        results.append(dict(model_name=names[ml.model_name], feature_name=features.get(ml.feature_name), processed_text=ml.processed_text, precision=precision, recall=recall, f1=f1))
    table_df = pd.DataFrame(results)
    table_df = table_df.reset_index(drop=True).set_index(['model_name', 'feature_name', 'processed_text'])

    if save:
        table_df.to_csv(save)
    return table_df

table = create_table(all_pipelines, os.path.join(BASE_PATH, 'testing_results_table.csv'))   
table

# Creating predictions on validation set (papers from Sep and Nov 2019)

This step can take several hours to run (took me aprox. 10 hours)

In [ ]:
PAPER_PATH = os.path.join('paper_download', 'out')
nov_papers = pd.read_json(os.path.join(PAPER_PATH, 'pubmed_papers-nov_1-30.json'))
sep_papers = pd.read_json(os.path.join(PAPER_PATH, 'pubmed_papers-sep_1-30.json'))

def get_text(paper_df, processed):
    if processed:
        return paper_df['processed_text'].values
    else:
        return paper_df['text'].values

predictions = {}

def fit_final_and_save(ml, filename):
    pipeline = ml.get_pipeline()
    X_cv, _, y_cv, _ = get_splits(df, ml.processed_text)
    pipeline = pipeline.fit(X_cv, y_cv) 
    directory = os.path.join(BASE_PATH, 'final_models')
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    path = os.path.join(directory, filename+'.pkl')
    with open(path, 'wb') as file:
        pickle.dump(pipeline, file)

    return pipeline

for ml in tqdm(all_pipelines[::-1]):
    nov_text = get_text(nov_papers, ml.processed_text)
    sep_text = get_text(sep_papers, ml.processed_text)

    filename = '%s-%s-%s' % (ml.model_name, ml.feature_name, ml.processed_text)
    if ml.model_name == 'card_shark':
        X_cv, _, y_cv, _ = get_splits(df, ml.processed_text)
        df_ = []
        for z in X_cv:
            if ml.processed_text:
                df_.append(df.loc[df['processed_text'] == z].copy())
            else:
                df_.append(df.loc[df['text'] == z].copy())
        df_ = pd.concat(df_, axis=0)
        df_ = df_.reset_index()
        exclude = ["in", "and", "the", "or", "if", "a", "of", "from", "by", "with"]
        processed = ml.processed_text
        (cardWords, cardJournals, cardAbstracts) = wordFrequency(df.loc[df['label']==1], exclude, process=processed) 
        (sampleWords, sampleJournals, sampleAbstracts) = wordFrequency(df_, exclude, process=processed)
        matrixA = matrixMaker(cardWords, sampleWords, True)
        matrixD = doubleMatrixMaker(cardAbstracts, sampleAbstracts, exclude)
        matrixJ = matrixMaker(cardJournals, sampleJournals, True)
        prediction_1 = bluePill('text_pmids', matrixA, matrixJ, matrixD, exclude, nov_papers, processed, True)
        prediction_2 = bluePill('text_pmids', matrixA, matrixJ, matrixD, exclude, sep_papers, processed, True)
        predictions[filename] = list(zip(nov_papers['pmid'], prediction_1)) + list(zip(sep_papers['pmid'], prediction_2))
    else:
        pipeline = fit_final_and_save(ml, filename)
        a = list(zip(nov_papers['pmid'], pipeline.predict(nov_text)))
        b = list(zip(sep_papers['pmid'], pipeline.predict(sep_text)))
        predictions[filename] = a + b

def convert(o):
    if isinstance(o, np.int32): return int(o)  
    raise TypeError

with open(os.path.join(BASE_PATH, 'all_predictions.json'), 'w') as file:
    json.dump(predictions, file, default=convert)    


## Parsing raw validation data

See `./validation_distribution` for methods used there.

In [ ]:
from validation_distribution import validation_visualization as vv
from importlib import reload  
reload(vv)

config = os.path.join('validation_distribution', 'resources', 'config.json')
prediction_path = os.path.join(BASE_PATH, 'all_predictions.json')
validation_out, b = vv.main(config, [prediction_path])

In [ ]:
final_total = {}
for _, item in validation_out.items():
    for method, results in item['acc'].items():
        for result_name, n in results.items():
            if method in final_total:
                final_total[method][result_name] += n
            else:
                final_total[method] = results
                break

## Stats on validation set

In [ ]:
validation_df = pd.DataFrame(final_total).transpose()/2 # Need to remove the duplicates from validation
validation_df['Model'] = [names.get(x.split('-')[0][6:]) for x in validation_df.index]
validation_df['Feature'] = [features.get(x.split('-')[1]) for x in validation_df.index]
validation_df['Processed text'] = [x.split('-')[-1] for x in validation_df.index]
validation_df = validation_df.reset_index(drop=True).set_index(['Model', 'Feature', 'Processed text'])
validation_df['Precision'] = validation_df['TP'] / (validation_df['TP'] + validation_df['FP'])
validation_df['Recall'] = validation_df['TP'] / (validation_df['TP'] + validation_df['FN'])
validation_df['f1'] = 2*validation_df['TP'] / (2*validation_df['TP'] + validation_df['FP'] + validation_df['FN'])
validation_df.to_csv(os.path.join(BASE_PATH, 'validation_results.csv'))
validation_df

## Stats on total +ve/-ve predictions

In [ ]:
def get_predictions():
    with open(os.path.join(BASE_PATH, 'all_predictions.json'), 'r') as file:
        all_predictions = json.load(file)

    out = []
    for name, results in all_predictions.items():
        model = names.get(name.split('-')[0])
        feature = features.get(name.split('-')[1])
        process = name.split('-')[-1] 
        positives = sum([1 for x in results if x[1] == 1])
        negatives = sum([1 for x in results if x[1] == 0])
        out.append(dict(model=model, feature=feature, process=process,positives=positives,negatives=negatives))
    return pd.DataFrame(out).reset_index(drop=True).set_index(['model', 'feature', 'process'])

validation_predictions = get_predictions()
validation_predictions.to_csv(os.path.join(BASE_PATH, 'model_sep_nov_predictions.csv'))

In [ ]:
validation_predictions

# Gridsearch hyperparameter tuning

Using the three chosen pipelines:

- logistic regression/TF-IDF word/processed
- naive bayes/word count/processed
- random forest/TF-IDF bi/tri-gram/processed


In [ ]:
C = np.logspace(-4, 4, 10)
PARAMETERS = dict(
    lr=[dict(
            lr__penalty=['l1'],
            lr__solver=['liblinear', 'saga'],
            lr__C=C,
            lr__max_iter=[10,50,100,150],
            lr__random_state=[100]
            ),
        dict(
            lr__penalty=['l2'],
            lr__solver=['newton-cg', 'lbfgs', 'sag', 'saga'],
            lr__C=C,
            lr__max_iter=[10,50,100,150],
            lr__random_state=[100]
            )
            ],

    rf=dict(
        rf__n_estimators = [int(x) for x in np.linspace(100, 1000, 5)][::-1],
        rf__max_features=['sqrt', 'log2'],
        rf__min_samples_leaf=[1, 4, 10, 30, 60],
        rf__random_state=[100]
        ),

    count_word={
        "count-word__max_df":[0.5,0.75,1],
        "count-word__max_features":[None, 5000, 10000, 50000]
    })

GRID_PATH = os.path.join('results', 'non-subsampled', 'final_models')
file_names = ['lr-tf_word-True.pkl', 'nb-count_word-True.pkl', 'rf-tf_bitrigram-True.pkl']
processed_splits = get_splits(df, True)
for file in file_names:
    PATH = os.path.join(GRID_PATH, file)
    with open(PATH, 'rb') as f:
        pl = pickle.load(f)
    steps = [step[0].replace('-', '_') for step in pl.steps]
    pl_parameters = {}
    for step_name, parameters in PARAMETERS.items():
        if step_name not in steps:
            continue
        if step_name == 'lr':
            pl_parameters = parameters
            [param.update(PARAMETERS['count_word']) for param in pl_parameters]
            break
        pl_parameters.update(parameters)
    grid_search = GridSearchCV(pl, pl_parameters, verbose=1, n_jobs=4, refit=True)
    grid_search.fit(processed_splits[0], processed_splits[2])
    estimator = grid_search.best_estimator_
    with open(os.path.join(GRID_PATH, 'grid_search_out-'+file), 'wb') as f:
        pickle.dump(estimator, f)

## Testing final models on hold-out data

In [ ]:
file_names = ['grid_search_out-lr-tf_word-True.pkl', 'grid_search_out-nb-count_word-True.pkl', 'grid_search_out-rf-tf_bitrigram-True.pkl']

def holdout_eval(pipeline, X_holdout, y_holdout):
    probas_ = pipeline.predict_proba(X_holdout)
    fpr, tpr, thresholds = metrics.roc_curve(
        y_holdout, probas_[:, 1])
    auc = metrics.auc(fpr, tpr)
    precision, recall, _ = metrics.precision_recall_curve(
        y_holdout, probas_[:, 1])

    predictions = pipeline.predict(X_holdout)
    p, r, f1, _ = metrics.precision_recall_fscore_support(y_holdout, predictions, average="weighted")
    cm = metrics.confusion_matrix(y_holdout, predictions)
    mse = metrics.mean_squared_error(y_holdout, predictions)

    results = dict(
        fpr=fpr,
        tpr=tpr,
        thresholds=thresholds,
        precision=precision,
        recall=recall,
        cm=cm,
        auc=auc,
        mse=mse,
        p=p,
        r=r,
        f1=f1,
        mean_fpr=np.linspace(0, 1, 100)
    )

    return results

processed_splits = get_splits(df, True)
holdout_results = []
for file_name in file_names:
    with open(os.path.join(GRID_PATH, file_name), 'rb') as f:
        pl = pickle.load(f)
    
    holdout_result = holdout_eval(pl, processed_splits[1], processed_splits[3])
    holdout_results.append((file_name, holdout_result))

In [ ]:
def create_roc(ml_items, save=False):
    plt.clf()
    fig = plt.figure(dpi=300)
    fig.set_figwidth(9)
    fig.set_figheight(8)
    ax = plt.axes(xlim=(-0.05,1.05), ylim=(-0.05,1.05))
    plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])
    plt.yticks([0.0,0.2,0.4,0.6,0.8,1.0])

    ax.plot([0,1], [0,1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    for (name, result) in ml_items:
        model_name = names.get(name.split('-')[1])
        feature_name = features.get(name.split('-')[2])
        color = colors.get(model_name)
        fpr = result['fpr']
        tpr = result['tpr']
        auc = result['auc']
        ax.plot(fpr, tpr, alpha=0.9, label=r'ROC %s, %s (AUC = %0.2f)' % (model_name, feature_name, auc), color=color)

    ax.legend(loc="lower right")
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')

    if save:
        plt.savefig(save+'.png')
        plt.savefig(save+'.svg', format="svg")

    plt.show()

create_roc(holdout_results, os.path.join('results', 'non-subsampled', 'grid_search_roc'))

In [ ]:
def create_table(ml_items, save=False):
    results = []
    for (name, result) in ml_items:
        model_name = names.get(name.split('-')[1])
        feature_name = features.get(name.split('-')[2])

        precision = result['p']
        recall = result['r']
        f1 = result['f1']
        results.append(dict(model_name=model_name, feature_name=feature_name, precision=precision, recall=recall, f1=f1))
    table_df = pd.DataFrame(results)
    table_df = table_df.reset_index(drop=True).set_index(['model_name', 'feature_name'])

    if save:
        table_df.to_csv(save)
    return table_df

table = create_table(holdout_results, os.path.join('results', 'non-subsampled', 'grid_search_holdout_results.csv'))   
table

## Retrospective comparison between CS2 and CS3

In [ ]:
PREDICTIONS_PATH = os.path.join('results', 'non-subsampled', 'retrospective_predictions')
PAPER_PATH = os.path.join('paper_download', 'out', 'jun-2017_to_dec-2020')
file_names = ['grid_search_out-lr-tf_word-True.pkl', 'grid_search_out-nb-count_word-True.pkl', 'grid_search_out-rf-tf_bitrigram-True.pkl']
PATH = os.path.join('results', 'non-subsampled', 'final_models')

In [ ]:
if not os.path.exists(PREDICTIONS_PATH):
    os.makedirs(PREDICTIONS_PATH)

for file_name in file_names:
    pl_predictions = set()
    with open(os.path.join(PATH, file_name), 'rb') as f:
        pl = pickle.load(f)
    
    for paper_file in os.listdir(PAPER_PATH):
        if not paper_file.endswith('.json'):
            continue
        papers = pd.read_json(os.path.join(PAPER_PATH, paper_file))

        predictions = pl.predict(papers['processed_text'])
        pl_predictions |= {pmid for pmid, pred in zip(papers['pmid'], predictions) if pred == 1}

    with open(os.path.join(PREDICTIONS_PATH, '%s-predictions.json' % file_name), 'w') as f:
        json.dump(list(pl_predictions), f)     

In [ ]:
TOTAL_PAPER_PMID_PATH = os.path.join('paper_download', 'out', 'jun-2017_to_dec-2020_pmids.json')

all_papers = set()

if os.path.exists(TOTAL_PAPER_PMID_PATH):
    with open(TOTAL_PAPER_PMID_PATH, 'r') as f:
        all_papers = set(json.load(f))

else:
    for paper_file in os.listdir(PAPER_PATH):
        if not paper_file.endswith('.json'):
            continue
        papers = pd.read_json(os.path.join(PAPER_PATH, paper_file))
        all_papers |= set(papers['pmid'])
    with open(TOTAL_PAPER_PMID_PATH, 'w') as f:
        json.dump(list(all_papers), f)

all_papers = {str(x) for x in all_papers}
print(len(all_papers))

In [ ]:
cs_3_predictions = {}
for file_name in os.listdir(PREDICTIONS_PATH):
    with open(os.path.join(PREDICTIONS_PATH, file_name), 'r') as f:
        predictions = json.load(f)
    name = file_name.split('-')[1]
    cs_3_predictions[name] = set(predictions)

for name, item in cs_3_predictions.items():
    cs_3_predictions[name] = {str(x) for x in item}

with open(os.path.join('retrospective_cs2_pred', 'out', 'cs_2_predictions.json'), 'r') as f:
    cs_2_predictions = json.load(f)

card_papers_path = 'card_pubs.csv'
card_papers = pd.read_csv(card_papers_path)
original_card_pmids = set(card_papers['accession'])
# Filter down to pmids found in the 3 million papers examined by the models
card_pmids = original_card_pmids.intersection(all_papers)
print(len(card_pmids))

In [ ]:
def create_retrospective_table(cs_3, cs_2, card, all_papers):
    cs_2_high = set(cs_2['high'])
    cs_2_low = set(cs_2['low'])
    PATH = os.path.join('results', 'non-subsampled')
    cs_2_only = (cs_2_high | cs_2_low) - all_papers

    cs_2_performance = len(cs_2_high.intersection(card) - cs_2_only)
    cs_2_ignored = len(cs_2_low.intersection(card) - cs_2_only - cs_2_high)

    rf = ('Random forest', len(all_papers), len(cs_3['rf']-cs_2_only), len(cs_3['rf'].intersection(card)-cs_2_only))
    nb = ('Naive Bayes', len(all_papers), len(cs_3['nb']-cs_2_only), len(cs_3['nb'].intersection(card)-cs_2_only))
    lr = ('Logistic regression', len(all_papers), len(cs_3['lr']-cs_2_only), len(cs_3['lr'].intersection(card)-cs_2_only))
    cs_2 = ('CARD:Shark 2', len((cs_2_high | cs_2_low) - cs_2_only), f'H: {len(cs_2_high - cs_2_only)} L: {len(cs_2_low - cs_2_only - cs_2_high)}', f'H: {cs_2_performance} L: {cs_2_ignored}')

    table_df = pd.DataFrame([lr, nb, rf, cs_2], columns=['Model name', 'Papers examined', 'Positive predictions', 'CARD overlap'])
    table_df = table_df.reset_index(drop=True).set_index(['Model name'])

    table_df.to_csv(os.path.join('results', 'non-subsampled', 'retrospec_results.csv'))

create_retrospective_table(cs_3_predictions, cs_2_predictions, card_pmids, all_papers)

In [ ]:
from venn import venn
import matplotlib.pyplot as plt

def create_venn(cs_3, cs_2, card, all_papers):
    cs_2_high = set(cs_2['high'])
    cs_2_low = set(cs_2['low'])
    PATH = os.path.join('results', 'non-subsampled')
    cs_2_only = (cs_2_high | cs_2_low) - all_papers
    v = {"Random forest":cs_3['rf'], "Naive Bayes":cs_3['nb'], "Logistic regression":cs_3['lr'], "CARD:Shark 2":cs_2_high - cs_2_only}
    venn(v)
    plt.savefig(os.path.join(PATH, 'venn_diagram_overall_overlap.png'))
    plt.show()

    cs_2_performance = cs_2_high.intersection(card) - cs_2_only
    rf_p = cs_3['rf'].intersection(card) - cs_2_only
    nb_p = cs_3['nb'].intersection(card) - cs_2_only
    lr_p = cs_3['lr'].intersection(card) - cs_2_only

    v2 = {"Random forest":rf_p, "Naive Bayes":nb_p, "Logistic regression":lr_p, "CARD:Shark 2":cs_2_performance}
    venn(v2)
    plt.savefig(os.path.join(PATH, 'venn_diagram_card_subset_overlap.png'))
    plt.show()


create_venn(cs_3_predictions, cs_2_predictions, card_pmids, all_papers)

In [ ]:
def cs_2_only(cs_2, card):
    cs_2_high = set(cs_2['high'])
    cs_2_low = set(cs_2['low'])
    cs_2_positive = (cs_2_high | cs_2_low).intersection(card)
    print(len(cs_2_positive))

cs_2_only(cs_2_predictions, card_pmids)